In [ ]:
# Now let's test the connection
from ai21 import AI21Client
import os

# paste your API key here or set it as an environment variable
API_KEY = os.environ.get("AI21_API_KEY")

client = AI21Client(api_key=API_KEY)

We will now create a Maestro run that uses Requirements

Remember Requirements should instruct Maestro how the output should be validated and fixed

Example requirements:
1. Don't mention anything about the company policy
2. Use a clear and formal tone
3. If the input contains a root cause analysis, include it

In [ ]:
run = client.beta.maestro.runs.create_and_poll(
    input="Write a product review for a smartphone",
    requirements=[
        {"name": "word_count", "description": "use 200-250 words"},
        {"name": "pros_and_cons", "description": "Include both pros and cons sections"},
        {"name": "rating", "description": "End with a 1-5 star rating. For example: (★★★★☆)"}
    ],
    budget="low",
    include=["requirements_result"],
)

print(run.status)
print(run.result)
print(run.requirements_result)


As you can see the request included 2 new parameters:
1. `budget` - The amount of effort Maestro will use to improve your result. We have 3 budget tiers:
    * `low` - Only validates the output and provides a report
    * `medium` - Validates and fixes the output
    * `high` - Uses more compute for output generation and fixing
2. `include` - This field tells Maestro to include certain fields in the output. In our case we included the `requirements_report` so that we can see the requirements scores in the output

Let's try a different example for text extraction

In [ ]:
input1 = """
You are an expert in extracting structured information from unstructured text, specifically travel booking conversations.
You are given a conversation as plain text between a user and an AI travel agent. Extract and return the following fields:
Name: Full name of the traveler
HotelName: Name of the hotel selected by the traveler
CheckInDate: Check-in date in YYYY-MM-DD format if the date can fully be extracted
CheckOutDate: Check-out date in YYYY-MM-DD format if the date can fully be extracted
City: City where the hotel is located
Country: Country where the hotel is located
NumberOfGuests: Total number of guests

Requirements:
1. Return a bulleted list where each field is a bullet
2. If any field is not mentioned or can’t be fully extracted from the conversation, fill it with "NA
3. Do not invent, infer, or assume any details that are not explicitly stated in the conversation.

Conversation:
User: Hi there! I'm planning a trip to Rome next month and need help booking a hotel.
AI: I'd be happy to help! Could you tell me your travel dates and how many people will be staying?
User: Sure. I'll be arriving on July 10th and checking out on July 15th. It'll be just me and my wife, so 2 guests.
AI: Great. Would you like me to recommend some hotels in Rome, Italy?
User: Yes, please.
AI: Here are a few options:
Hotel Artemide
The Rome EDITION
Hotel Nazionale
Do any of these interest you?
User: Hotel Artemide sounds perfect. Let's go with that.
AI: Excellent choice! May I have your full name for the reservation?
User: It's Jonathan Rivera.
AI: Thank you, Jonathan. I've reserved a room at Hotel Artemide in Rome, Italy for 2 guests from July 10 to July 15.
"""

run = client.beta.maestro.runs.create_and_poll(
    input=input1,
)

print(run.id)
print(run.result)
print(run.requirements_result)


Uusing MVO

In [ ]:
input2 = """
You are an expert in extracting structured information from unstructured text, specifically travel booking conversations.
You are given a conversation as plain text between a user and an AI travel agent. Extract and return the following fields:
Name: Full name of the traveler
HotelName: Name of the hotel selected by the traveler
CheckInDate: Check-in date in YYYY-MM-DD format if the date can fully be extracted
CheckOutDate: Check-out date in YYYY-MM-DD format if the date can fully be extracted
City: City where the hotel is located
Country: Country where the hotel is located
NumberOfGuests: Total number of guests

Conversation:
User: Hi there! I'm planning a trip to Rome next month and need help booking a hotel.
AI: I'd be happy to help! Could you tell me your travel dates and how many people will be staying?
User: Sure. I'll be arriving on July 10th and checking out on July 15th. It'll be just me and my wife, so 2 guests.
AI: Great. Would you like me to recommend some hotels in Rome, Italy?
User: Yes, please.
AI: Here are a few options:
Hotel Artemide
The Rome EDITION
Hotel Nazionale
Do any of these interest you?
User: Hotel Artemide sounds perfect. Let's go with that.
AI: Excellent choice! May I have your full name for the reservation?
User: It's Jonathan Rivera.
AI: Thank you, Jonathan. I've reserved a room at Hotel Artemide in Rome, Italy for 2 guests from July 10 to July 15.
"""

"""

Do not invent, infer, or assume any details that are not explicitly stated in the conversation.
return a bulleted list where each field is a bullet
"""

requirements = [
    {"name": "NA", "description": "If any field is not mentioned or can’t be fully extracted from the conversation, fill it with 'NA'"},
    {"name": "Do not invent", "description": "Do not invent, infer, or assume any details that are not explicitly stated in the conversation."},
    {"name": "bulleted list", "description": "return a bulleted list where each field is a bullet"},
]

run = client.beta.maestro.runs.create_and_poll(
    input=input2,
    requirements=requirements,
    budget="medium",
    include=["requirements_result"],
)

print(run.id)
print(run.result)
print(run.requirements_result)